# Acceso de vehículos ResiPark (Detección) - Python 3.11.13

In [1]:
!python --version

Python 3.11.13


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Instalar dependencias

In [3]:
!nvidia-smi

Thu Sep 25 02:15:26 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   46C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install timm -q
!pip install accelerate -q
!pip install einops -q
!pip install fastapi uvicorn nest_asyncio
!pip install ultralytics opencv-python-headless
!apt-get install unzip wget -y
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu124

In [5]:
import torch

if torch.cuda.is_available():
    print("CUDA está disponible. Usando GPU:", torch.cuda.get_device_name(0))
else:
    print("CUDA no está disponible. Usando CPU.")

CUDA está disponible. Usando GPU: Tesla T4


In [ ]:
import os



In [7]:
!mkdir my_models
!mkdir my_models/Florence_2

#### Florence Model

In [ ]:
from transformers import AutoProcessor
from transformers import AutoModelForCausalLM, AutoProcessor

model = AutoModelForCausalLM.from_pretrained("microsoft/Florence-2-large",
                                             cache_dir="/content/my_models/Florence_2",
                                             device_map="cuda",
                                             trust_remote_code=True,
                                             attn_implementation="eager")


processor = AutoProcessor.from_pretrained(
    "microsoft/Florence-2-large",  # ID del repositorio
    token = os.environ["HF_TOKEN"],
    trust_remote_code = True,
    cache_dir = "/content/my_models/Florence_2"  # Se guardará en esta carpeta
)

#### App YOLOv11 + OCR

In [9]:
import base64
from io import BytesIO
from PIL import Image
from fastapi import FastAPI, UploadFile, File
from fastapi.responses import JSONResponse
import uvicorn
from ultralytics import YOLO
import cv2
import numpy as np
import re
import logging
from typing import List, Dict, Any, Optional, Tuple
from contextlib import asynccontextmanager
from PIL import Image

# Configurar logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Variables globales
model_plates = None
model_cars = None
prompt = '<OCR>'

@asynccontextmanager
async def lifespan(app: FastAPI):
    # Startup
    logger.info("[INFO] Cargando modelos YOLO...")
    global model_plates, model_cars, prompt


    model_plates = YOLO("/content/drive/MyDrive/Proyecto/10k_50_plates.pt") #cambiar direccion de los pesos
    model_cars = YOLO('/content/drive/MyDrive/Proyecto/10k_50_cars.pt') # cambiar direccion de los pesos

    logger.info("[INFO] Cargando Florence-2...")

    logger.info("[INFO] API lista para recibir peticiones")

    yield  # Aquí se ejecuta durante la vida de la app

app = FastAPI(lifespan=lifespan)

def get_detection_zone(img: np.ndarray) -> Tuple[int, int, int, int]:
    """
    Define zona rectangular de detección (ROI)
    """
    h, w, _ = img.shape

    # Zona central inferior (donde suelen circular los autos)
    width_margin_percent = 0.1
    height_start_percent = 0.4
    height_end_percent = 0.9

    margin_w = int(w * width_margin_percent)
    x1 = margin_w
    x2 = w - margin_w

    y1 = int(h * height_start_percent)
    y2 = int(h * height_end_percent)

    return x1, y1, x2, y2

def boxes_intersect(box1: List[int], box2: List[int]) -> bool:
    """
    Verifica si dos bounding boxes se intersectan (se tocan)

    Args:
        box1: [x1, y1, x2, y2] del vehículo
        box2: [x1, y1, x2, y2] del ROI

    Returns:
        True si hay intersección, False otherwise
    """
    x1_veh, y1_veh, x2_veh, y2_veh = box1
    x1_roi, y1_roi, x2_roi, y2_roi = box2

    # Verificar intersección en eje X
    intersect_x = not (x2_veh < x1_roi or x1_veh > x2_roi)

    # Verificar intersección en eje Y
    intersect_y = not (y2_veh < y1_roi or y1_veh > y2_roi)

    # Hay intersección si ambos ejes se intersectan
    return intersect_x and intersect_y

def calculate_intersection_area(box1: List[int], box2: List[int]) -> int:
    """
    Calcula el área de intersección entre dos bounding boxes
    """
    x1_veh, y1_veh, x2_veh, y2_veh = box1
    x1_roi, y1_roi, x2_roi, y2_roi = box2

    # Coordenadas de la intersección
    x1_int = max(x1_veh, x1_roi)
    y1_int = max(y1_veh, y1_roi)
    x2_int = min(x2_veh, x2_roi)
    y2_int = min(y2_veh, y2_roi)

    # Calcular área (asegurarse de que no sea negativa)
    width = max(0, x2_int - x1_int)
    height = max(0, y2_int - y1_int)

    return width * height

def is_vehicle_in_roi(vehicle_box: List[int], roi_box: List[int], min_intersection: int = 100) -> bool:
    """
    Verifica si un vehículo toca el ROI con al menos un área mínima de intersección

    Args:
        vehicle_box: [x1, y1, x2, y2] del vehículo
        roi_box: [x1, y1, x2, y2] del ROI
        min_intersection: área mínima de intersección en píxeles (default: 100)

    Returns:
        True si el vehículo toca el ROI con al menos el área mínima
    """
    if not boxes_intersect(vehicle_box, roi_box):
        return False

    # Calcular área de intersección
    intersection_area = calculate_intersection_area(vehicle_box, roi_box)

    # Verificar que la intersección sea significativa (evitar toques de 1 píxel)
    return intersection_area >= min_intersection


def prueba_ocr(texts):

  texts = texts.upper()
  texts = re.sub(r" +", "", texts)
  texts = texts.strip('\n').split()

  candidates = []

  for t in texts:

        # limpiar todo lo que no sea letras o números
        t = re.sub(r"[^A-Z0-9]", "", t)

        # validaciones básicas
        if len(t) < 5 or len(t) > 8:  # ahora permitimos 5–8
            continue
        if t.isalpha() or t.isdigit():  # solo letras o solo números
            continue

        # regex básica: mezcla de letras y números
        if not re.match(r"^(?=.*[A-Z])(?=.*[0-9])[A-Z0-9]+$", t):
            continue

        candidates.append(t)

  if not candidates:
      return None

    # heurística: preferimos el string más largo (más completo)
  plate = max(candidates, key=len)

  return plate


def florence_ocr(prompt, image_):
  '''
  Modelo Florence-2
  Transcripcion y llamado de funcion para limpiar
  '''

  if isinstance(image_, np.ndarray):
      image = Image.fromarray(image_).convert("RGB")

  else:
      image = Image.open(image_).convert("RGB")

  inputs = processor(text=prompt, images=image, return_tensors="pt").to("cuda:0")

  generated_ids = model.generate(
      input_ids=inputs["input_ids"],
      pixel_values=inputs["pixel_values"],
      max_new_tokens=32,
  	  early_stopping=True,
  	  do_sample=False,
  	  num_beams=5
    )

  generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
  parsed_answer = processor.post_process_generation(generated_text, task=prompt, image_size=(image.width, image.height))

  return prueba_ocr(parsed_answer['<OCR>'])


@app.post("/predict")
async def predict(file: UploadFile = File(...), min_intersection: int = 100):
    try:
        contents = await file.read()
        npimg = np.frombuffer(contents, np.uint8)
        img = cv2.imdecode(npimg, cv2.IMREAD_COLOR)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

        detections = []
        roi_zone = get_detection_zone(img_rgb) # Zona de detección

        vehicle_results = model_cars(img_rgb, conf=0.7, verbose=False)
        vehicle_boxes = vehicle_results[0].boxes

        if vehicle_boxes is None:
            return JSONResponse(content={"detections": []})

        boxes = vehicle_boxes.xyxy.cpu().numpy()
        confs = vehicle_boxes.conf.cpu().numpy()
        class_ids = vehicle_boxes.cls.cpu().numpy().astype(int)

        for i, box in enumerate(boxes):
            x1, y1, x2, y2 = map(int, box[:4])
            vehicle_box = [x1, y1, x2, y2]

            if not is_vehicle_in_roi(vehicle_box, roi_zone, min_intersection):
                continue

            vehicle_crop = img_rgb[y1:y2, x1:x2]
            if vehicle_crop.size == 0:
                continue

            # OPTIMIZACIÓN: Placas con menor resolución
            plate_results = model_plates(vehicle_crop, conf=0.7, verbose=False)
            plate_boxes = plate_results[0].boxes

            if plate_boxes is None:
                continue

            plate_confs = plate_boxes.conf.cpu().numpy()

            # VERIFICAR QUE HAY CONFIANZAS VÁLIDAS
            if plate_confs.size == 0 or len(plate_confs) == 0:
                continue  # No hay valores de confianza

            best_plate_idx = np.argmax(plate_confs)
            plate_box = plate_boxes.xyxy.cpu().numpy()[best_plate_idx]
            px1, py1, px2, py2 = map(int, plate_box[:4])

            # Padding
            h, w, _ = vehicle_crop.shape
            pad = 0.08
            x1_pad = max(0, int(px1 - (px2 - px1) * pad))
            y1_pad = max(0, int(py1 - (py2 - py1) * pad))
            x2_pad = min(w, int(px2 + (px2 - px1) * pad))
            y2_pad = min(h, int(py2 + (py2 - py1) * pad))


            plate_crop = vehicle_crop[y1_pad:y2_pad, x1_pad:x2_pad]
            if plate_crop.size == 0:
                continue

            # Florence-2 (OCR)
            plate_text = florence_ocr(prompt, plate_crop)

            if plate_text == None:
                continue

            detections.append({
                "vehicle_type": model_cars.names[class_ids[i]],
                "vehicle_confidence": round(float(confs[i]), 3),
                "plate_confidence": round(float(plate_confs[best_plate_idx]), 3),
                "plate_text": plate_text,
            })


        return JSONResponse(content={"detections": detections})

    except Exception as e:
        logger.error(f"Error en procesamiento: {str(e)}")
        return JSONResponse(content={"detections": []})

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


#### Uvicorn server LocalHost

In [10]:
import threading

def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

thread = threading.Thread(target=run, daemon=True)
thread.start()

#### Tunel CloudFlare

In [11]:
!cloudflared tunnel --url http://127.0.0.1:8000 --no-autoupdate

2025-09-25T02:23:37Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-09-25T02:23:37Z INF Requesting new quick Tunnel on trycloudflare.com...


INFO:     Application startup complete.


2025-09-25T02:23:40Z INF +--------------------------------------------------------------------------------------------+
2025-09-25T02:23:40Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-09-25T02:23:40Z INF |  https://characterized-consciousness-functioning-effectively.trycloudflare.com             |
2025-09-25T02:23:40Z INF +--------------------------------------------------------------------------------------------+
2025-09-25T02:23:40Z INF Cannot determine default configuration path. No file [config.yml config.yaml] in [~/.cloudflared ~/.cloudflare-warp ~/cloudflare-warp /etc/cloudflared /usr/local/etc/cloudflared]
2025-09-25T02:23:40Z INF Version 2025.9.1 (Checksum 3dc1dc4252eae3c691861f926e2b8640063a2ce534b07b7a3f4ec2de439ecfe3)
2025-09-25T02:23:40Z INF GOOS: linux, GOVersion: go1.24.4, GoArch: amd64
2025-09-25T02:23:40Z INF Settings: map[ha-connections:1 no-autoupdate:true protocol:quic url:http://127.0.0.1:8000]
2025-09-2

INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


INFO:     2800:150:125:1a1c:1ca3:988d:9b93:e778:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2800:150:125:1a1c:1ca3:988d:9b93:e778:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2800:150:125:1a1c:1ca3:988d:9b93:e778:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2800:150:125:1a1c:1ca3:988d:9b93:e778:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2800:150:125:1a1c:1ca3:988d:9b93:e778:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2800:150:125:1a1c:1ca3:988d:9b93:e778:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2800:150:125:1a1c:1ca3:988d:9b93:e778:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2800:150:125:1a1c:1ca3:988d:9b93:e778:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2800:150:125:1a1c:1ca3:988d:9b93:e778:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2800:150:125:1a1c:1ca3:988d:9b93:e778:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2800:150:125:1a1c:1ca3:988d:9b93:e778:0 - "POST /predict HTTP/1.1" 200 OK
INFO:     2800:150:125:1a1c:1ca3:988d:9b93:e778:0 - "POST /predict HTTP/1.1"